# Embedding And Storage:

## Install Qdrant and Required Libraries:

In [ ]:
#!pip install protobuf==5.26.1
!pip install qdrant-client
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently tak

## Set Up and Configure Qdrant:



### Initialize the Qdrant Client:

In [ ]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct

client = QdrantClient(":memory:")

### Create a Collection in Qdrant:

In [ ]:
if client.collection_exists("chatbot_embeddings_test"):
    # drop collection if already exist
    client.delete_collection("chatbot_embeddings_test")

In [ ]:
client.recreate_collection(
    collection_name="chatbot_embeddings_test",
    vectors_config=models.VectorParams(size=384,
                                       distance=models.Distance.DOT),
)

<ipython-input-10-1d0421a09bb2>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

### Generate the Embeddings:

In [ ]:
from sentence_transformers import SentenceTransformer

# load a model that produces 384-dimensional embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# examples of text chunks that talk about illnesses
text_chunks = ["""Hypertension is a condition in which the force of
               the blood against the artery walls is too high. It
               is also known as high blood pressure and can lead to
                serious healthcomplications if left untreated.""",
               """Diabetes is a chronic condition characterized by
               high levels of sugar in the blood. Common symptoms
               include excessive thirst, frequent urination, extreme
               fatigue, and blurred vision.""",
               """Heart disease refers to a range of conditions that
               affect the heart, including coronary artery disease,
               heart rhythm problems, and heart defects. Major causes
               include high cholesterol, high blood pressure,
               smoking, and obesity.""",
               """Asthma is a respiratory condition that causes the
               airways to become inflamed and narrowed. Treatment
               typically includes the use of inhalers that contain
               bronchodilators and corticosteroidsto reduce
               inflammation and open the airways."""]

# generate embeddings
embeddings = model.encode(text_chunks)
print(embeddings.shape)  # ensuring that number of chunks is 384


(4, 384)


### Load the Embeddings into Qdrant:

In [ ]:
for i, embedding in enumerate(embeddings):
    client.upsert(
        collection_name="chatbot_embeddings_test",
        points=[
            PointStruct(
                id= i+1,
                vector= embedding.tolist(),
                payload= {"text": text_chunks[i]}
            )
        ]
    )

### Test the Retrieval Process:

In [ ]:
import time

query = "What are the common symptoms associated with diabetes?"

query_embedding = model.encode(query).tolist()

start_time = time.time()

search_results = client.search(
    collection_name="chatbot_embeddings_test",
    query_vector=query_embedding,
    limit=4  # Number of top results to return
)

end_time = time.time()
print(f"Query took {end_time - start_time} seconds")

for result in search_results:
    print(f"Score: {result.score}, Text: {result.payload['text']}")

Query took 0.0074880123138427734 seconds
Score: 0.7462586693829891, Text: Diabetes is a chronic condition characterized by high levels of sugar in the blood. Common symptoms include excessive thirst, frequent urination, extreme fatigue, and blurred vision.
Score: 0.20300906244734218, Text: Heart disease refers to a range of conditions that affect the heart, including coronary artery disease, heart rhythm problems, and heart defects. Major causes include high cholesterol,
               high blood pressure, smoking, and obesity.
Score: 0.11358719147429977, Text: Hypertension is a condition in which the force of the blood against the artery walls is too high. It is also known as high blood pressure and can lead to serious health
                    complications if left untreated.
Score: -0.050635168511529896, Text: Asthma is a respiratory condition that causes the airways to become inflamed and narrowed. Treatment typically includes the use of inhalers that contain bronchodilators and c

In [ ]:
query = "What are the possible treatment for asthma?"

query_embedding = model.encode(query).tolist()

start_time = time.time()

search_results = client.search(
    collection_name="chatbot_embeddings_test",
    query_vector=query_embedding,
    limit=4  # Number of top results to return
)

end_time = time.time()
print(f"Query took {end_time - start_time} seconds")

for result in search_results:
    print(f"Score: {result.score}, Text: {result.payload['text']}")

Query took 0.0004870891571044922 seconds
Score: 0.6926184422358225, Text: Asthma is a respiratory condition that causes the airways to become inflamed and narrowed. Treatment typically includes the use of inhalers that contain bronchodilators and corticosteroids
               to reduce inflammation and open the airways.
Score: 0.13331556723786356, Text: Heart disease refers to a range of conditions that affect the heart, including coronary artery disease, heart rhythm problems, and heart defects. Major causes include high cholesterol,
               high blood pressure, smoking, and obesity.
Score: 0.0005567263951216025, Text: Hypertension is a condition in which the force of the blood against the artery walls is too high. It is also known as high blood pressure and can lead to serious health
                    complications if left untreated.
Score: -0.043613061214324525, Text: Diabetes is a chronic condition characterized by high levels of sugar in the blood. Common symptoms includ